# Plotting live data from Cyton Board using Python and pyqtgraph


## Objective

1. Use the pyOpenBCI library to aquire data from the OpenBCI board and plot it in real time.
2. Process the OpenBCI data to get the different EEG bands (Alpha, Beta, etc..)


### Why use pyqtgraph?

From the pyqtgraph website:

"PyQtGraph is a graphics and user interface library for Python that provides functionality commonly required in engineering and science applications. Its primary goals are 1) to provide fast, interactive graphics for displaying data (plots, video, etc.) and 2) to provide tools to aid in rapid application development (for example, property trees such as used in Qt Designer).

For plotting, pyqtgraph is not nearly as complete/mature as matplotlib, but runs much faster. Matplotlib is more aimed toward making publication-quality graphics, whereas pyqtgraph is intended for use in data acquisition and analysis applications."

Because the Cyton Board data comes at a rate of 250Hz, a fast way to display data is needed and pyqtgraph is the way to go.


### Notes:
1. This notebook is being run on a Windows 10 Laptop, using a Python3 environment.

## First let's import the necessary packages:


In [1]:
import sys
sys.path.append('C:/Python37/Lib/site-packages')

from IPython.display import clear_output
%matplotlib qt5
%matplotlib inline


from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import random
from pyOpenBCI import OpenBCICyton
import threading
import time
import numpy as np
from scipy import signal

#### Now let's try to run the [test code](https://github.com/OpenBCI/pyOpenBCI) from the pyOpenBCI repo

We are doing this to make sure all the dependencies are installed correctly. When working this code should print the raw data of the board. If everything is working correctly you can interrupt the kernel to exit. You should see the Cyton data printing.

We are going to run the test script for .02 seconds to check that everything is working correctly. (This time might vary depending on your set up)

In [13]:
from pyOpenBCI import OpenBCICyton

def print_raw(sample):
    print(sample.channels_data)

board = OpenBCICyton(port='COM5', daisy=False)

def start_cyton():
    try:
        board.start_stream(print_raw)
    except:
        pass

y = threading.Thread(target=start_cyton)
y.daemon = True
y.start()

time.sleep(.02)
board.disconnect()

Serial established
Skipped 25 bytes before start found
[-8388608, -8388608, -8388608, 8388607, -8388608, -8291119, -8293139, -8268097]
[-8388608, -8388608, -8388608, 8388607, -8352848, -8285641, -8287507, -8273677]
[-8388608, -8388608, -8388608, 8388607, -8330673, -8284409, -8285959, -8271927]
[-8388608, -8388608, -8388608, 8388607, -8388608, -8289629, -8291256, -8266523]
Closing Serial


### Let's now add filters to process the data

#### What do we need?

1. A notch filter at 60Hz to filter out main power interferance. 
2. A highpass filter at 1Hz to 50Hz in order to remove the DC offset.
3. Get an FFT of the processed data.
4. Take an average of the FFT data to get the EEG bands.
5. Plot everything live using pyqtgraph.


In [6]:
SCALE_FACTOR = (4500000)/24/(2**23-1) #From the pyOpenBCI repo
colors = 'rgbycmwr'

# Set up GUI Layout
app = QtGui.QApplication([])
win = pg.GraphicsWindow(title='Python OpenBCI GUI')
ts_plots = [win.addPlot(row=i, col=0, colspan=2, title='Channel %d' % i, labels={'left': 'uV'}) for i in range(1,9)]
fft_plot = win.addPlot(row=1, col=2, rowspan=4, title='FFT Plot', labels={'left': 'uV', 'bottom': 'Hz'})
fft_plot.setLimits(xMin=1,xMax=125, yMin=0, yMax=1e7)
waves_plot = win.addPlot(row=5, col=2, rowspan=4, title='EEG Bands', labels={'left': 'uV', 'bottom': 'EEG Band'})
waves_plot.setLimits(xMin=0.5, xMax=5.5, yMin=0)
waves_xax = waves_plot.getAxis('bottom')
waves_xax.setTicks([list(zip(range(6), ('', 'Delta', 'Theta', 'Alpha', 'Beta', 'Gama')))])
data = [[0,0,0,0,0,0,0,0]]

# Define OpenBCI callback function
def save_data(sample):
    global data
    data.append([i*SCALE_FACTOR for i in sample.channels_data])

# Define function to update the graphs
def updater():
    global data, plots, colors
    t_data = np.array(data[-1250:]).T #transpose data
    fs = 250 #Hz

    # Notch Filter
    def notch_filter(val, data, fs=250):
        notch_freq_Hz = np.array([float(val)])
        for freq_Hz in np.nditer(notch_freq_Hz):
            bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])
            b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
            fin = data = signal.lfilter(b, a, data)
        return fin

    # Bandpass filter
    def bandpass(start, stop, data, fs = 250):
        bp_Hz = np.array([start, stop])
        b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
        return signal.lfilter(b, a, data, axis=0)

    # Applying the filters
    nf_data = [[],[],[],[],[],[],[],[]]
    bp_nf_data = [[],[],[],[],[],[],[],[]]

    for i in range(8):
        nf_data[i] = notch_filter(60, t_data[i])
        bp_nf_data[i] = bandpass(15, 80, nf_data[i])

    # Plot a time series of the raw data
    for j in range(8):
        ts_plots[j].clear()
        ts_plots[j].plot(pen=colors[j]).setData(t_data[j])

    # Get an FFT of the data and plot it
    sp = [[],[],[],[],[],[],[],[]]
    freq = [[],[],[],[],[],[],[],[]]
    
    fft_plot.clear()
    for k in range(8):
        sp[k] = np.absolute(np.fft.fft(bp_nf_data[k]))
        freq[k] = np.fft.fftfreq(bp_nf_data[k].shape[-1], 1.0/fs)
        fft_plot.plot(pen=colors[k]).setData(freq[k], sp[k])


    # Define EEG bands
    eeg_bands = {'Delta': (1, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band (Only consider first channel)
    eeg_band_fft = dict()
    sp_bands = np.absolute(np.fft.fft(t_data[1]))
    freq_bands = np.fft.fftfreq(t_data[1].shape[-1], 1.0/fs)

    for band in eeg_bands:
        freq_ix = np.where((freq_bands >= eeg_bands[band][0]) &
                           (freq_bands <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(sp_bands[freq_ix])

    # Plot EEG Bands
    bg1 = pg.BarGraphItem(x=[1,2,3,4,5], height=[eeg_band_fft[band] for band in eeg_bands], width=0.6, brush='r')
    waves_plot.clear()
    waves_plot.addItem(bg1)

# Define thread function
def start_board():
    board = OpenBCICyton(port='COM5', daisy=False)
    board.start_stream(save_data)
    
# Initialize Board and graphing update
if __name__ == '__main__':
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        x = threading.Thread(target=start_board)
        x.daemon = True
        x.start()

        timer = QtCore.QTimer()
        timer.timeout.connect(updater)
        timer.start(0)


        QtGui.QApplication.instance().exec_()

The code above should produce a GUI similar to the one below. You can change the parameters of the GUI to fit your needs and also use the code above as a starting point to make your own projects.

<img src="GUI_python.JPG">

Let's make sure we disconnect from the board when we are done.

In [15]:
board.disconnect()